IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\FAULT_M1_10


VOLO M1 FAULT 10%

In [3]:
rcou_m1_fault10 = pd.read_csv("RCOU.csv")
rcou_m1_fault10 = rcou_m1_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m1_fault10 = rcou_m1_fault10[((rcou_m1_fault10['C9'] >= 1350) & (rcou_m1_fault10['C10'] >= 1350) & (rcou_m1_fault10['C11'] >= 1350) & (rcou_m1_fault10['C12'] >= 1350) & (rcou_m1_fault10['C13'] >= 1350) & (rcou_m1_fault10['C14']>= 1350))]
display(rcou_m1_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
591,1648547241,1431,1424,1373,1477,1370,1479
592,1648558959,1351,1504,1451,1414,1366,1493
594,1648579931,1467,1420,1429,1459,1448,1441
596,1648600801,1525,1374,1393,1511,1466,1443
597,1648611573,1436,1483,1442,1478,1413,1504
...,...,...,...,...,...,...,...
9746,1749459761,1576,1423,1531,1477,1501,1508
9747,1749470298,1555,1447,1515,1492,1459,1545
9748,1749480739,1516,1484,1534,1465,1440,1556
9749,1749491176,1555,1431,1525,1465,1478,1513


In [4]:
xkf1_m1_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault10 = xkf1_m1_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault10 = xkf1_m1_fault10[((xkf1_m1_fault10['TimeUS'] >= 1648547241) & (xkf1_m1_fault10['TimeUS'] <= 1749502709	))]
xkf1_m1_fault10 = xkf1_m1_fault10.reset_index(drop=True)
print(xkf1_m1_fault10)

          TimeUS  Roll  Pitch    Yaw
0     1648557084 -1.24   1.59  19.96
1     1648567530 -1.34   1.57  19.96
2     1648577440 -1.25   1.57  19.94
3     1648588414 -1.13   1.58  19.94
4     1648598616 -1.03   1.62  19.93
...          ...   ...    ...    ...
9157  1749457351 -1.53   0.06   6.15
9158  1749468272 -1.50   0.07   6.16
9159  1749478311 -1.41   0.00   6.13
9160  1749489170 -1.21  -0.05   6.14
9161  1749500410 -0.92  -0.12   6.20

[9162 rows x 4 columns]


In [5]:
att_m1_fault10 = pd.read_csv("ATT.csv")
att_m1_fault10 = att_m1_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault10 = att_m1_fault10[((att_m1_fault10['TimeUS'] >= 1648547241) & (att_m1_fault10['TimeUS'] <= 1749502709	))]
att_m1_fault10=att_m1_fault10.reset_index(drop=True)
print(att_m1_fault10)

          TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     1648558171    -0.06 -1.24     -0.40   1.59   19.79  19.96
1     1648568640    -0.05 -1.34     -0.41   1.57   19.79  19.96
2     1648579278    -0.04 -1.25     -0.41   1.57   19.79  19.94
3     1648589763    -0.04 -1.13     -0.42   1.58   19.79  19.94
4     1648600108    -0.04 -1.03     -0.43   1.62   19.79  19.93
...          ...      ...   ...       ...    ...     ...    ...
9158  1749459114     0.74 -1.53     -0.32   0.06    8.12   6.15
9159  1749469562     0.75 -1.50     -0.34   0.07    8.12   6.16
9160  1749479836     0.76 -1.41     -0.34   0.00    8.12   6.13
9161  1749490199     0.75 -1.21     -0.33  -0.05    8.12   6.14
9162  1749501809     0.74 -0.92     -0.32  -0.12    8.12   6.20

[9163 rows x 7 columns]


In [6]:
if len(att_m1_fault10)>len(xkf1_m1_fault10):
    to_add=att_m1_fault10[len(xkf1_m1_fault10):]
    att_m1_fault10=att_m1_fault10[:len(xkf1_m1_fault10)]

if len(xkf1_m1_fault10)>len(att_m1_fault10):
    to_add=xkf1_m1_fault10[len(att_m1_fault10):]

for idx,i in enumerate(att_m1_fault10['Roll']):
    if(xkf1_m1_fault10['Roll'][idx] != i):
        att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Pitch']):
    if(xkf1_m1_fault10['Pitch'][idx] != i):
        att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault10['Yaw']):
    if(xkf1_m1_fault10['Yaw'][idx] != i):
        att_m1_fault10['Yaw'][idx] = ((att_m1_fault10['Yaw'][idx] + xkf1_m1_fault10['Yaw'][idx])/2)

att_m1_fault10=pd.concat([att_m1_fault10,to_add])

print(att_m1_fault10)

C:\Users\chiar\AppData\Local\Temp\ipykernel_28856\4117696472.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault10['Roll'][idx] = ((att_m1_fault10['Roll'][idx] + xkf1_m1_fault10['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28856\4117696472.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault10['Pitch'][idx] = ((att_m1_fault10['Pitch'][idx] + xkf1_m1_fault10['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28856\4117696472.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

          TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0     1648558171    -0.06 -1.240     -0.40  1.590   19.79  19.960
1     1648568640    -0.05 -1.340     -0.41  1.570   19.79  19.960
2     1648579278    -0.04 -1.250     -0.41  1.570   19.79  19.940
3     1648589763    -0.04 -1.130     -0.42  1.580   19.79  19.940
4     1648600108    -0.04 -1.030     -0.43  1.620   19.79  19.930
...          ...      ...    ...       ...    ...     ...     ...
9158  1749459114     0.74 -1.515     -0.32  0.065    8.12   6.155
9159  1749469562     0.75 -1.455     -0.34  0.035    8.12   6.145
9160  1749479836     0.76 -1.310     -0.34 -0.025    8.12   6.135
9161  1749490199     0.75 -1.065     -0.33 -0.085    8.12   6.170
9162  1749501809     0.74 -0.920     -0.32 -0.120    8.12   6.200

[9163 rows x 7 columns]


In [7]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault10 = esc_0_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault10 = esc_0_m1_fault10[((esc_0_m1_fault10['TimeUS'] >= 1648547241) & (esc_0_m1_fault10['TimeUS'] <= 1749502709))]
esc_0_m1_fault10=esc_0_m1_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault10 = esc_1_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault10 = esc_1_m1_fault10[((esc_1_m1_fault10['TimeUS'] >= 1648547241) & (esc_1_m1_fault10['TimeUS'] <= 1749502709))]
esc_1_m1_fault10=esc_1_m1_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault10 = esc_2_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault10 = esc_2_m1_fault10[((esc_2_m1_fault10['TimeUS'] >= 1648547241) & (esc_2_m1_fault10['TimeUS'] <= 1749502709))]
esc_2_m1_fault10=esc_2_m1_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault10 = esc_3_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault10 = esc_3_m1_fault10[((esc_3_m1_fault10['TimeUS'] >= 1648547241) & (esc_3_m1_fault10['TimeUS'] <= 1749502709))]
esc_3_m1_fault10=esc_3_m1_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault10 = esc_4_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault10 = esc_4_m1_fault10[((esc_4_m1_fault10['TimeUS'] >= 1648547241) & (esc_4_m1_fault10['TimeUS'] <= 1749502709))]
esc_4_m1_fault10=esc_4_m1_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault10 = esc_5_m1_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault10 = esc_5_m1_fault10[((esc_5_m1_fault10['TimeUS'] >= 1648547241) & (esc_5_m1_fault10['TimeUS'] <= 1749502709))]
esc_5_m1_fault10=esc_5_m1_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [8]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault10 = imu_0_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault10 = imu_0_m1_fault10[((imu_0_m1_fault10['TimeUS'] >= 1648547241) & (imu_0_m1_fault10['TimeUS'] <= 1749502709))]

imu_1_m1_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault10 = imu_1_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault10 = imu_1_m1_fault10[((imu_1_m1_fault10['TimeUS'] >= 1648547241) & (imu_1_m1_fault10['TimeUS'] <= 1749502709))]

imu_2_m1_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault10 = imu_2_m1_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault10 = imu_2_m1_fault10[((imu_2_m1_fault10['TimeUS'] >= 1648547241) & (imu_2_m1_fault10['TimeUS'] <= 1749502709))]

imu_m1_fault10 = pd.concat((imu_0_m1_fault10, imu_1_m1_fault10, imu_2_m1_fault10))
imu_m1_fault10=imu_m1_fault10.groupby(imu_m1_fault10.TimeUS, as_index=False).mean()

display(imu_m1_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,1648552405,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406
1,1648558068,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221
2,1648561764,0.041824,0.100781,0.001137,0.231455,0.761186,-9.726832
3,1648563543,0.220820,0.071107,0.001237,0.379594,0.636334,-9.590402
4,1648566595,0.112270,-0.002903,0.002685,0.414795,0.359073,-9.528167
...,...,...,...,...,...,...,...
34809,1749488276,0.280210,-0.035147,0.002372,0.501059,0.538888,-10.258573
34810,1749490094,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337
34811,1749492809,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707
34812,1749499200,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693


In [9]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault10 = pd.merge_ordered(imu_m1_fault10,att_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_0_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_1_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_2_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_3_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_4_m1_fault10)
m1_fault10 = pd.merge_ordered(m1_fault10, esc_5_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")
m1_fault10 = pd.merge_ordered(m1_fault10, rcou_m1_fault10)
m1_fault10=m1_fault10.fillna(method="ffill").fillna(method="bfill")

In [10]:
index_to_remove=[]
for ind in att_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m1_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

9163
10011
10872
11707
12544
13396
14239


In [11]:
m1_fault10 = m1_fault10[~m1_fault10.TimeUS.isin(index_to_remove)]
m1_fault10["TimeUS"] = m1_fault10["TimeUS"] - m1_fault10.iloc[0]["TimeUS"]
m1_fault10["TimeUS"] = m1_fault10["TimeUS"].astype(int)
m1_fault10["Guasto"] = 2

In [12]:
from datetime import timedelta
m1_fault10=m1_fault10.reset_index(drop=True)
m1_fault10['TimeUS'] = pd.to_datetime(m1_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.038953,0.004357,0.030866,0.365539,0.468139,-9.389406,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2
1,00:00:00.005663,-0.223346,0.002401,0.015638,0.191140,0.485832,-9.585221,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1431.0,1424.0,1373.0,1477.0,1370.0,1479.0,2
2,00:00:00.009359,0.041824,0.100781,0.001137,0.231455,0.761186,-9.726832,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1351.0,1504.0,1451.0,1414.0,1366.0,1493.0,2
3,00:00:00.011138,0.220820,0.071107,0.001237,0.379594,0.636334,-9.590402,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1351.0,1504.0,1451.0,1414.0,1366.0,1493.0,2
4,00:00:00.014190,0.112270,-0.002903,0.002685,0.414795,0.359073,-9.528167,-0.06,-1.240,-0.40,...,2.48,4410.0,3.01,1351.0,1504.0,1451.0,1414.0,1366.0,1493.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34809,00:01:40.935871,0.280210,-0.035147,0.002372,0.501059,0.538888,-10.258573,0.76,-1.310,-0.34,...,2.18,5600.0,3.06,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2
34810,00:01:40.937689,0.245654,-0.019537,0.029526,0.445560,0.603047,-10.375337,0.76,-1.310,-0.34,...,2.18,5600.0,3.06,1516.0,1484.0,1534.0,1465.0,1440.0,1556.0,2
34811,00:01:40.940404,0.280214,-0.001078,0.047156,0.174956,0.641513,-10.555707,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2
34812,00:01:40.946795,0.296264,-0.050650,0.081374,-0.083817,0.647380,-10.938693,0.75,-1.065,-0.33,...,2.18,5600.0,3.06,1555.0,1431.0,1525.0,1465.0,1478.0,1513.0,2


## Analisi nel tempo e in frequenza

In [13]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [14]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault10)/350)):
        V1=m1_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [15]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = 2
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28856\80979898.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28856\80979898.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.025278,0.017191,-0.117391,0.133346,79.0,76.0,3.609819,3.564965,-0.021368,...,5781.210237,3.194017,0.188088,-1.070037,3.223243,119.0,118.0,12.541374,10.763319,2
1,1.0,-0.025284,0.017142,-0.109234,0.133164,107.0,87.0,2.692896,2.653739,-0.020985,...,5813.733886,3.196250,0.189307,-1.066487,3.225645,126.0,125.0,12.851213,11.042833,2
2,2.0,-0.025410,0.017099,-0.104507,0.133027,110.0,92.0,2.793322,2.182012,-0.020225,...,5810.014592,3.198470,0.190509,-1.063391,3.228031,145.0,144.0,13.144419,11.282863,2
3,3.0,-0.026125,0.017232,-0.127146,0.133661,78.0,75.0,2.853510,2.746864,-0.019520,...,5771.977143,3.200678,0.191695,-1.060714,3.230401,120.0,119.0,13.420816,11.480551,2
4,4.0,-0.026810,0.017350,-0.150134,0.134237,81.0,82.0,3.404802,2.920552,-0.019146,...,5604.258678,3.202873,0.192864,-1.058426,3.232757,131.0,130.0,13.680267,11.633432,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,94.0,-0.049459,0.016614,-0.416218,0.137924,109.0,111.0,3.810675,2.865564,0.006946,...,9777.636167,3.397258,0.307436,-1.009808,3.442108,152.0,149.0,11.325783,9.349819,2
95,95.0,-0.049956,0.016687,-0.426220,0.138365,103.0,99.0,4.248018,3.142264,0.007303,...,9588.435683,3.399484,0.308955,-1.010496,3.444525,145.0,142.0,11.729749,9.020589,2
96,96.0,-0.050214,0.016679,-0.428853,0.138432,108.0,117.0,4.997842,2.740228,0.007535,...,9388.905168,3.401700,0.310457,-1.011353,3.446930,150.0,147.0,12.107483,8.648223,2
97,97.0,-0.050774,0.016782,-0.435845,0.139007,122.0,131.0,4.855823,2.726305,0.007858,...,9179.420856,3.403906,0.311942,-1.012367,3.449322,149.0,146.0,12.458605,8.234021,2


In [16]:
path_file = path_file.replace(r"M1\FAULT_M1_10", "")
os.chdir(path_file)
m1_fault10.to_csv('m1_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
